In [ ]:
# --- CONFIGURATION ---
TOKEN = "38ah1K39qUM1jmReDrQejideYwA_VXNZzC69PhzACHVnbdMm"

print("🧹 [1/5] Deep Cleaning System...")
!rm -rf /etc/apt/sources.list.d/r2u.list
!apt-get update -qq > /dev/null
!apt-get install -y docker.io adb libgbm1 -qq > /dev/null

print("⚡ [2/5] Starting Engine...")
import os, subprocess, time, threading
get_ipython().system_raw('dockerd --storage-driver=vfs --host=unix:///var/run/docker.sock &')
time.sleep(15)

print("📱 [3/5] Booting Android...")
!docker run -d --privileged -p 5555:5555 redroid/redroid:11.0.0-latest > /dev/null

print("🌐 [4/5] Starting Web Viewer (Port 7860)...")
# We use a more stable way to launch the viewer
os.system('npx -y ws-scrcpy-docker --port 7860 > viewer_log.txt 2>&1 &')

print("⏳ Waiting for Viewer to wake up...")
time.sleep(10) # Give the website time to actually start

print("🔗 [5/5] Generating Ngrok Link...")
from pyngrok import ngrok
try:
    ngrok.kill()
    ngrok.set_auth_token(TOKEN)
    public_url = ngrok.connect(7860).public_url
    print("\n" + "="*50)
    print(f"✅ SUCCESS! CLICK HERE: {public_url}")
    print("="*50)
    print("Note: If you see a 'Connection Refused' error, just REFRESH the page after 10 seconds.")
except Exception as e:
    print(f"❌ Link Error: {e}")